In [1]:
import torch
import os

print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name(s):", [torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())])
# Get Slurm-allocated CPU count or fallback
n_jobs = int(os.environ.get("SLURM_CPUS_PER_TASK", os.cpu_count()))
print("Number of jobs:", n_jobs)
n_human = int(n_jobs/4)

CUDA available: True
Device count: 1
Device name(s): ['NVIDIA GeForce RTX 3050 6GB Laptop GPU']
Number of jobs: 16


In [ ]:
params = {
    'model': 'lorenz',
    'case': 'synthetic_lorenz',
    'input_dim': 80,
    'latent_dim': 3,
    'poly_order': 2,
    'include_sine': False,
    'fix_coefs': False,
    'svd_dim': 40,
    'delay_embedded': True,
    'scale': True,
    'coefficient_initialization': 'constant',
    "coefficient_initial_value": 1e-2,
    'widths_ratios': [1.0, 0.6],

    # Training
    'max_epochs': 500,
    'patience': 20,
    'batch_size': 2**9,
    'learning_rate': 1e-3,
    "lr_decay": 0.999,

    # Loss Weights
    'loss_weight_rec': 0.3,
    'loss_weight_sindy_z': 0.001,
    'loss_weight_sindy_x': 0.001,
    'loss_weight_sindy_regularization': 1e-5,
    'loss_weight_integral': 0.1,
    'loss_weight_x0': 0.01,
    'loss_weight_layer_l2': 0.0,
    'loss_weight_layer_l1': 0.0,

    # SINDy
    'coefficient_threshold': 1e-5,
    'threshold_frequency': 5,
    'print_frequency': 10,
    'sindy_pert': 0.0,
    'ode_net': False,
    'ode_net_widths': [1.5, 2.0],
    'exact_features': True,
    'use_bias': True,

    # Misc
    'train_ratio': 0.8,
    'data_path': './trained_models',
    'save_checkpoints': False,
    'save_freq': 5,
    'learning_rate_sched': False,
    'use_sindycall': False,
    'sparse_weighting': None,
    'system_coefficients': None
}

In [ ]:
import torch
import time
from TrainingCode.TrainingCode import TrainModel  # adjust import path if needed
from ParamOptCode.DefaultParams import params
from AdditionalFunctionsCode.SolverFunctions import SynthData
from DataGenerationCode.DataGeneration import LorenzSystem

# === SETUP ===
params["max_epochs"] = 500
params["input_dim"] = params["svd_dim"]  # ensure consistent dimensions
params["batch_size"] = 2**9  # larger batch to leverage GPU better
params["dt"] = 0.01 

# === Generate synthetic data ===
lorenz_system = LorenzSystem()
synth_data = SynthData(input_dim=params["input_dim"])
z0 = torch.tensor([1.0, 1.0, 1.0])
synth_data.run_sim(tend=32, dt=0.01, z0=z0.numpy(), apply_svd=True, svd_dim=params["svd_dim"])

# === Choose device ===
# Change to 'cpu' to benchmark that version
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")  # Uncomment to test CPU explicitly

# === Train ===
trainer = TrainModel(synth_data, params, device=device)

start = time.time()
trainer.fit()
end = time.time()

device_type = "GPU" if device.type == "cuda" else "CPU"
print(f"\n=== Training on {device_type} ({device}) completed in {(end - start)/60:.2f} minutes ===")

In [ ]:
print(params["svd_dim"])

Perform the Hyperparameter Optimization

In [2]:
from ParamOptCode.ParameterOptimizer import HyperparameterOptimizer
from ParamOptCode.DefaultParams import params 
import time
#os.remove('C:/Users/mehme/OneDrive/Desktop/All/Code/LorenzReclaim/optuna_study.db')
start_time = time.time()
device = torch.device("cpu")
device_type = "GPU" if device.type == "cuda" else "CPU"
params["input_dim"] = params["svd_dim"]
optimizer = HyperparameterOptimizer(n_trials=1, n_jobs=1)
best = optimizer.run()
end_time = time.time()
print(f"Best hyperparameters: {best}")
print(f"Hyperparameter optimization completed in {(end_time - start_time)/60:.2f} minutes")

[I 2025-04-26 15:22:26,698] Using an existing study with name 'sindy_opt' instead of creating a new one.


Generating Lorenz system...
self.x (rec_v): (3120, 10)
self.dx: (3120, 10)

Active Loss Weights:
loss_weight_rec: 0.6622743745527734
loss_weight_sindy_z: 0.016239880477356985
loss_weight_sindy_x: 0.02321062610095071
loss_weight_sindy_regularization: 2.7619615198903407e-05
loss_weight_integral: 0.014287323354111776
loss_weight_x0: 0.0007395865706237733
loss_weight_layer_l2: 0.006573024585874463
loss_weight_layer_l1: 0.009866992557136803
Epoch 0: Loss = 6.616799 | LR = 0.003076
Epoch 10: Loss = 6.503344 | LR = 0.002853
Epoch 20: Loss = 6.452836 | LR = 0.002645
Epoch 30: Loss = 6.417363 | LR = 0.002453
Epoch 40: Loss = 6.383078 | LR = 0.002275
Epoch 50: Loss = 6.275696 | LR = 0.002110
Epoch 60: Loss = 6.027059 | LR = 0.001956
Epoch 70: Loss = 5.575470 | LR = 0.001814
Epoch 80: Loss = 5.351608 | LR = 0.001682
Epoch 90: Loss = 5.140810 | LR = 0.001560
Epoch 100: Loss = 5.113349 | LR = 0.001447
Epoch 110: Loss = 5.098843 | LR = 0.001342
Epoch 120: Loss = 5.091484 | LR = 0.001244
Epoch 130: L

[I 2025-04-26 15:26:04,662] Trial 17 finished with value: 19.57076072692871 and parameters: {'loss_weight_rec': 0.6622743745527734, 'loss_weight_sindy_z': 0.016239880477356985, 'loss_weight_sindy_x': 0.02321062610095071, 'loss_weight_sindy_regularization': 2.7619615198903407e-05, 'loss_weight_integral': 0.014287323354111776, 'loss_weight_x0': 0.0007395865706237733, 'loss_weight_layer_l2': 0.006573024585874463, 'loss_weight_layer_l1': 0.009866992557136803, 'learning_rate': 0.003099528863125862, 'lr_decay': 0.9924839788755989, 'batch_size': 512, 'coefficient_threshold': 1.0146608312377015e-05}. Best is trial 17 with value: 19.57076072692871.



Final SINDy Model:
dz0/dt = -0.0819 * 1 + -0.0038 * z0 + 1.2147 * z1 + -1.0114 * z2 + -0.5789 * z0z0 + -0.4838 * z0z1 + 0.4089 * z0z2 + 1.4212 * z1z1 + -0.3229 * z1z2 + -1.5367 * z2z2
dz1/dt = -0.4544 * 1 + -1.3458 * z0 + -1.1389 * z1 + 0.3821 * z2 + -0.4371 * z0z0 + -1.5701 * z0z1 + -1.9060 * z0z2 + -1.1196 * z1z1 + 0.4910 * z1z2 + 1.1011 * z2z2
dz2/dt = 0.6766 * 1 + 1.4738 * z0 + -0.4503 * z1 + 1.2039 * z2 + 0.5049 * z0z0 + 1.9343 * z0z1 + 1.3843 * z0z2 + -0.9846 * z1z1 + -0.2789 * z1z2 + 1.3961 * z2z2

Top 5 Trials by Static Loss:

📌 Trial 1 (Global Trial #17):
  Static Loss: 19.570761
  Params:
    loss_weight_rec: 0.6622743745527734
    loss_weight_sindy_z: 0.016239880477356985
    loss_weight_sindy_x: 0.02321062610095071
    loss_weight_sindy_regularization: 2.7619615198903407e-05
    loss_weight_integral: 0.014287323354111776
    loss_weight_x0: 0.0007395865706237733
    loss_weight_layer_l2: 0.006573024585874463
    loss_weight_layer_l1: 0.009866992557136803
    learning_rate: